In [93]:
import numpy as np
from sklearn import preprocessing, neighbors, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import pandas as pd
from statistics import mean 

In [94]:
df = pd.read_csv('College_Sleep_Study.csv')
#remove outliers
df.replace('?', -99999, inplace = True)

In [95]:
print(df)

     Gender  ClassYear  LarkOwl  NumEarlyClass  EarlyClass   GPA  \
0         0          4  Neither              0           0  3.60   
1         0          4  Neither              2           1  3.24   
2         0          4      Owl              0           0  2.97   
3         0          1     Lark              5           1  3.76   
4         0          4      Owl              0           0  3.20   
5         1          4  Neither              0           0  3.50   
6         1          2     Lark              2           1  3.35   
7         0          2     Lark              0           0  3.00   
8         0          1  Neither              2           1  4.00   
9         0          4  Neither              2           1  2.90   
10        1          2  Neither              1           1  3.70   
11        1          2  Neither              0           0  3.00   
12        0          1  Neither              4           1  3.30   
13        1          3  Neither              2  

In [96]:
df.corr()

,Gender,ClassYear,NumEarlyClass,EarlyClass,GPA,ClassesMissed,CognitionZscore,PoorSleepQuality,DepressionScore,AnxietyScore,...,Happiness,Drinks,WeekdayBed,WeekdayRise,WeekdaySleep,WeekendBed,WeekendRise,WeekendSleep,AverageSleep,AllNighter
Gender,1.000000,-0.076144,-0.099747,-0.080713,-0.244577,0.131141,-0.206471,0.027084,0.028844,-0.131882,...,-0.047498,0.396170,0.237235,0.190288,-0.063556,0.178281,0.188284,0.044907,-0.036440,0.195909
ClassYear,-0.076144,1.000000,-0.268725,-0.180652,-0.154805,0.135365,-0.001435,-0.010596,-0.048064,0.040517,...,0.057582,0.051503,-0.002674,-0.010236,-0.021448,-0.021891,-0.018202,0.010426,-0.013987,-0.114867
NumEarlyClass,-0.099747,-0.268725,1.000000,0.804650,0.101380,-0.082841,0.001013,-0.094431,0.023021,-0.028370,...,-0.049671,-0.119709,-0.179649,-0.468227,-0.217952,-0.151908,0.012299,0.139762,-0.132150,-0.037746
EarlyClass,-0.080713,-0.180652,0.804650,1.000000,0.079487,-0.096245,0.097946,-0.089490,-0.052908,-0.086181,...,-0.003961,-0.072934,-0.153591,-0.418200,-0.227246,-0.106632,-0.002098,0.115056,-0.150049,-0.038694
GPA,-0.244577,-0.154805,0.101380,0.079487,1.000000,-0.181543,0.266822,-0.052230,0.011918,0.015644,...,0.002940,-0.269305,-0.129505,-0.213778,-0.079768,-0.122166,-0.165884,0.020946,-0.060633,-0.057702
ClassesMissed,0.131141,0.135365,-0.082841,-0.096245,-0.181543,1.000000,-0.157074,0.298799,0.153940,0.173969,...,-0.100381,0.078302,0.214166,0.180374,-0.118511,0.239280,0.237040,0.051150,-0.081582,0.142903
CognitionZscore,-0.206471,-0.001435,0.001013,0.097946,0.266822,-0.157074,1.000000,-0.086733,-0.065403,-0.065759,...,0.039394,-0.155855,-0.059647,-0.110563,-0.023068,-0.105262,-0.099905,-0.009554,-0.023854,-0.061254
PoorSleepQuality,0.027084,-0.010596,-0.094431,-0.089490,-0.052230,0.298799,-0.086733,1.000000,0.433692,0.451310,...,-0.349951,-0.001990,0.224763,0.022364,-0.241611,0.156046,0.078541,-0.023985,-0.218726,0.104454
DepressionScore,0.028844,-0.048064,0.023021,-0.052908,0.011918,0.153940,-0.065403,0.433692,1.000000,0.486040,...,-0.542265,-0.011469,-0.005350,-0.043299,-0.047774,0.032244,0.084430,0.060270,-0.017170,0.064138
AnxietyScore,-0.131882,0.040517,-0.028370,-0.086181,0.015644,0.173969,-0.065759,0.451310,0.486040,1.000000,...,-0.354938,-0.057705,0.064981,-0.052562,-0.158815,0.006393,-0.050739,-0.044802,-0.155568,0.079018


In [97]:
#some algorithm require a categorical target feature while others accept a continuous value.
#make 2 df one with a categorical feature and one with a continuous value.
#loop through gpa to convert it to categorical
for i in range(0,253):
    if (df.at[i,'GPA'] >= 0.0) & (df.at[i,'GPA'] < 1.0):
       df.set_value(i, "GPA", int(0))
    elif (df.at[i,'GPA'] >= 1.0) & (df.at[i,'GPA'] <2.0):
        df.set_value(i, "GPA", int(1))
    elif (df.at[i,'GPA'] >= 2.0) & (df.at[i,'GPA'] <3.0):
        df.set_value(i, "GPA", int(2))
    else:
        df.set_value(i, "GPA", int(3))

C:\Users\Chase\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
C:\Users\Chase\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [24]:
df.corr()

,Gender,ClassYear,NumEarlyClass,EarlyClass,GPA,ClassesMissed,CognitionZscore,PoorSleepQuality,DepressionScore,AnxietyScore,...,Happiness,Drinks,WeekdayBed,WeekdayRise,WeekdaySleep,WeekendBed,WeekendRise,WeekendSleep,AverageSleep,AllNighter
Gender,1.000000,-0.076144,-0.099747,-0.080713,-0.239993,0.131141,-0.206471,0.027084,0.028844,-0.131882,...,-0.047498,0.396170,0.237235,0.190288,-0.063556,0.178281,0.188284,0.044907,-0.036440,0.195909
ClassYear,-0.076144,1.000000,-0.268725,-0.180652,-0.076822,0.135365,-0.001435,-0.010596,-0.048064,0.040517,...,0.057582,0.051503,-0.002674,-0.010236,-0.021448,-0.021891,-0.018202,0.010426,-0.013987,-0.114867
NumEarlyClass,-0.099747,-0.268725,1.000000,0.804650,0.129000,-0.082841,0.001013,-0.094431,0.023021,-0.028370,...,-0.049671,-0.119709,-0.179649,-0.468227,-0.217952,-0.151908,0.012299,0.139762,-0.132150,-0.037746
EarlyClass,-0.080713,-0.180652,0.804650,1.000000,0.093804,-0.096245,0.097946,-0.089490,-0.052908,-0.086181,...,-0.003961,-0.072934,-0.153591,-0.418200,-0.227246,-0.106632,-0.002098,0.115056,-0.150049,-0.038694
GPA,-0.239993,-0.076822,0.129000,0.093804,1.000000,-0.078963,0.144038,-0.069283,-0.002479,-0.027682,...,0.006705,-0.235506,-0.098977,-0.117983,-0.012233,-0.087850,-0.079843,-0.000601,-0.010785,-0.000340
ClassesMissed,0.131141,0.135365,-0.082841,-0.096245,-0.078963,1.000000,-0.157074,0.298799,0.153940,0.173969,...,-0.100381,0.078302,0.214166,0.180374,-0.118511,0.239280,0.237040,0.051150,-0.081582,0.142903
CognitionZscore,-0.206471,-0.001435,0.001013,0.097946,0.144038,-0.157074,1.000000,-0.086733,-0.065403,-0.065759,...,0.039394,-0.155855,-0.059647,-0.110563,-0.023068,-0.105262,-0.099905,-0.009554,-0.023854,-0.061254
PoorSleepQuality,0.027084,-0.010596,-0.094431,-0.089490,-0.069283,0.298799,-0.086733,1.000000,0.433692,0.451310,...,-0.349951,-0.001990,0.224763,0.022364,-0.241611,0.156046,0.078541,-0.023985,-0.218726,0.104454
DepressionScore,0.028844,-0.048064,0.023021,-0.052908,-0.002479,0.153940,-0.065403,0.433692,1.000000,0.486040,...,-0.542265,-0.011469,-0.005350,-0.043299,-0.047774,0.032244,0.084430,0.060270,-0.017170,0.064138
AnxietyScore,-0.131882,0.040517,-0.028370,-0.086181,-0.027682,0.173969,-0.065759,0.451310,0.486040,1.000000,...,-0.354938,-0.057705,0.064981,-0.052562,-0.158815,0.006393,-0.050739,-0.044802,-0.155568,0.079018


In [98]:
#all fields must be numerical remove non numerical features and target feature
#string type LarkOwl, AlcoholUse, DepressionStatus, AnxietyStatus, Stress
x = np.array(df.drop(['LarkOwl', 'AlcoholUse', 'DepressionStatus', 'AnxietyStatus', 'Stress', 'GPA'],1))
#df of only target feature
y = np.array(df['GPA'])

In [105]:
''''Want to write a for loop here that will iterate through train_test_split, build models and get accuracy x times.
Save the accuracies to arrays so we can get the average accuracy for each model'''
clf1acc = []
clf2acc = []
clf3acc = []

for i in range(0,100):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.4)
    
    clf1 = neighbors.KNeighborsClassifier()
    clf1.fit(x_train, y_train)
    accuracy = clf1.score(x_test, y_test)
    clf1acc.append(accuracy)

    clf2 = DecisionTreeClassifier()
    clf2.fit(x_train, y_train)
    accuracy = clf2.score(x_test, y_test)
    clf2acc.append(accuracy)
    
    #Support Vector Classifier
    clf3 = SVC(gamma='auto')
    clf3.fit(x_train, y_train)
    accuracy = clf3.score(x_test, y_test)
    clf3acc.append(accuracy)
    
print('K Nearest Neighbors Average Accuracy: ', mean(clf1acc))
print('Decision Tree Classifier Average Accuracy: ', mean(clf2acc))
print('Support Vector Classifier Average Accuracy: ', mean(clf3acc))


K Nearest Neighbors Average Accuracy:  0.7566666666666667
Decision Tree Classifier Average Accuracy:  0.6757843137254902
Support Vector Classifier Average Accuracy:  0.796078431372549


In [71]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.4)

In [ ]:
'''
function calls for other algorithms!


clf = svm.SVR()
clf.fit(trainingData, trainingScores)
print("SVR")
print(clf.predict(predictionData))

clf = LogisticRegression()
clf.fit(trainingData, trainingScores)
print("LogisticRegression")
print(clf.predict(predictionData))

clf = DecisionTreeClassifier()
clf.fit(trainingData, trainingScores)
print("DecisionTreeClassifier")
print(clf.predict(predictionData))

clf = KNeighborsClassifier()
clf.fit(trainingData, trainingScores)
print("KNeighborsClassifier")
print(clf.predict(predictionData))

clf = LinearDiscriminantAnalysis()
clf.fit(trainingData, trainingScores)
print("LinearDiscriminantAnalysis")
print(clf.predict(predictionData))

clf = GaussianNB()
clf.fit(trainingData, trainingScores)
print("GaussianNB")
print(clf.predict(predictionData))

clf = SVC()
clf.fit(trainingData, trainingScores)
print("SVC")
print(clf.predict(predictionData))'''